In [2]:
from cloudvolume import CloudVolume, view
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cc3d
from tifffile.tifffile import imwrite
from FANC_auto_recon.segmentation import rootID_lookup
%pwd

'/workspaces/FANC_auto_recon/nuclei_prediction'

For now, I'll only look at a tiny region of entire dataset. So we need to think about how to scale up this approach later.

In [3]:
image_cv = CloudVolume(
    'precomputed://gs://lee-lab_female-adult-nerve-cord_tokyo-storage/alignmentV4/em/rechunked',
    progress=False,
    cache=True, # cache to disk to avoid repeated downloads
    use_https=True
)

nuclei_cv = CloudVolume(
    'precomputed://gs://ranl_scratch_zetta_30/neuroglancer/nuclei/3d528bc3def9f51e09ae55e68ee46518',
    progress=True,
    cache=True, # cache to disk to avoid repeated downloads
    use_https=True
)

In [4]:
image = image_cv.download_point( (52785, 86885, 1894), mip=4, size=(128, 128, 256) ) # mip2+ only
# mip needs to be the same?
# 4.3*2^4*128/45=196
# 30s

In [5]:
nuclei = nuclei_cv.download_point( (52785, 86885, 1894), mip=[68.8,68.8,45.0], size=(128, 128, 256) ) # mip 0 or 4 only
# 10min!?

Downloading: 100%|██████████| 162/162 [00:02<00:00,  3.73it/s]

In [ ]:
# view(image)

In [ ]:
# view(nuclei)

In [6]:
test = image[:,:,:]
testn_temp = nuclei[:,:,:]
testn = np.where(testn_temp > 0.5, 1, 0)
type(testn)

numpy.ndarray

In [6]:
view(nuclei)
# 9 cell bodies

# plt.imshow(testn_temp[:,:,149])
# plt.imshow(testn[:,:,149])

Viewer server listening to http://localhost:8080

Continuing program execution...


In [7]:
# print(testn.ndim) 
# 4
# print(testn.size) 
# print(testn.shape) 
# (128, 128, 256, 1)
testn_s = np.squeeze(testn)
print(testn_s.shape) 

(128, 128, 256)


In [8]:
# 26-connectivity
# https://en.wikipedia.org/wiki/Pixel_connectivity
# https://github.com/seung-lab/connected-components-3d
labels_in = testn_s
labels_out, N = cc3d.connected_components(labels_in, return_N=True, connectivity=26) # free
print(N)
# 15

15


It has to be 9, but 15...

In [31]:
# write numpy array into tiff using tifffile
# pip install tifffile
volume = testn_s.swapaxes(0, 2).astype('float32')
imwrite('../Output/testn_s.tif', volume, imagej=True)

In [9]:
def bbox2_3D(img):

    x = np.any(img, axis=(1, 2))
    y = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    xmin, xmax = np.where(x)[0][[0, -1]]
    ymin, ymax = np.where(y)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return xmin, xmax, ymin, ymax, zmin, zmax

In [10]:
list=[]
for segid in range(1, N+1):
  extracted_image = labels_out * (labels_out == segid)
  bbox = bbox2_3D(extracted_image)
  list.append(bbox)

print(list)

[(99, 127, 0, 27, 0, 72), (7, 50, 21, 62, 0, 6), (118, 127, 60, 95, 0, 12), (26, 111, 106, 127, 0, 76), (11, 119, 24, 105, 92, 222), (106, 127, 0, 28, 160, 240), (98, 116, 39, 49, 212, 235), (0, 64, 0, 18, 219, 255), (115, 127, 34, 45, 221, 239), (0, 0, 12, 16, 223, 228), (98, 101, 53, 62, 232, 235), (102, 127, 108, 127, 232, 255), (84, 122, 28, 84, 236, 255), (91, 98, 38, 46, 242, 245), (92, 93, 56, 59, 243, 244)]


In [12]:
x_thres = 33 # 50/(4.3*2^4/45) = 50/1.53
y_thres = 33
z_thres = 50

In [20]:
list2=[]
for segid in range(0, N):
  X = list[segid][1] - list[segid][0]
  Y = list[segid][3] - list[segid][2]
  Z = list[segid][5] - list[segid][4]
  if X >= x_thres and Y >= y_thres and Z >= z_thres:
    center = ((list[segid][1] + list[segid][0])/2,
      (list[segid][3] + list[segid][2])/2,
      (list[segid][5] + list[segid][4])/2)
    list2.append(center)
  else:
    pass

print(list2)

[(65.0, 64.5, 157.0)]


In [19]:
shift =  (list2) -(128/2, 128/2, 256/2)
coordinates + shift = center of this soma??

need to change to mip0 here

72

In [ ]:
from FANC_auto_recon.segmentation import rootID_lookup as IDlookup

IDlookup.segIDs_from_pts_cv(pts,
                       cv,
                       n=100000,
                       max_tries = 3,return_roots=True):

In [ ]:
image_cv.cache.flush()
nuclei_cv.cache.flush() 

In [ ]:
xmax-xmin
ymax-ymin
zmax-zmin